In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *

Sticker_datasources = ['event=BBM-STICKER-RECEIVED','event=BBM-STICKER_SPONSORED_LINK-CLICK','event=BBM-STICKER-CLICK','event=BBM-STICKER-DOWNLOAD','event=BBM-STICKER-SEND']
for ds in Sticker_datasources:
    #reading in sticker type data
    feature_data = spark.read.parquet("gs://ds-url-catag/plenty_stickers_data/"+ds+"/derived_features/Final/")
    main_data = feature_data
    for cn in main_data.columns:
        #if users can pay or not
        if "sum_paid" in cn:
            main_data = main_data.withColumn("Can_Pay",when(main_data[cn] !=0 ,"yes").otherwise("No"))
            #Ability to pay for users
            # calculate average sticker pack download to distinguish the clusters 
            #categorize ability to pay
        if "sticker_packs" in cn:
            main_data = main_data.withColumn('Abilty_to_pay',when(main_data[cn] >= 43.0,'Very High') \
                                             .otherwise(when((main_data[cn] >= 7.00) & (main_data[cn] < 43.0),'High') \
                                             .otherwise(when((main_data[cn] > 3.0) & (main_data[cn] < 7.0),'Medium') \
                                             .otherwise(when((main_data[cn] > 1.4) & (main_data[cn] < 3.0),'low') \
                                             .otherwise('very low')))))
        #get the percentiles for free,paid and subscribed among the clusters
        #usage rate of free,paid and subscribed
        if "distinct" in cn:
            distinct_column_name = cn
        if "sum_free" in cn:
            main_data = main_data.withColumn('Freestickerusagerate',main_data[cn]/main_data[distinct_column_name])
        if "sum_paid" in cn:
            main_data = main_data.withColumn('paidstickerusagerate',main_data[cn]/main_data[distinct_column_name])
        if "sum_subs" in cn:
            main_data = main_data.withColumn('subsstickerusagerate',main_data[cn]/main_data[distinct_column_name])

    from pyspark.sql import Window
    #Categorizing the usage rates 
    # #calculate the top percent rank for users performing better in the same cluster 
    window_freerank = Window.partitionBy('Abilty_to_pay').orderBy(main_data['Freestickerusagerate'].desc())
    main_data = main_data.withColumn('Free_Stickers_rank',percent_rank().over(window_freerank).alias('rank'))

    window_paidrank = Window.partitionBy('Abilty_to_pay').orderBy(main_data['paidstickerusagerate'].desc())
    main_data = main_data.withColumn('Paid_Stickers_rank',percent_rank().over(window_freerank).alias('rank'))

    window_subsrank = Window.partitionBy('Abilty_to_pay').orderBy(main_data['subsstickerusagerate'].desc())
    main_data = main_data.withColumn('Subscribed_Stickers_rank',percent_rank().over(window_freerank).alias('rank'))

    #assign low,medium and high values
    main_data = main_data.withColumn('Free_sticker_usage',when(col('Free_Stickers_rank') <= 0.2,'High').otherwise(when( (col('Free_Stickers_rank') > 0.2) & (col('Free_Stickers_rank') < 0.5),'medium').otherwise('low')))
    main_data = main_data.withColumn('paid_sticker_usage',when(col('Paid_Stickers_rank') <= 0.2,'High').otherwise(when( (col('Paid_Stickers_rank') > 0.2) & (col('Paid_Stickers_rank') < 0.5),'medium').otherwise('low')))
    main_data = main_data.withColumn('Subscribed_sticker_usage',when(col('Subscribed_Stickers_rank') <= 0.2,'High').otherwise(when( (col('Subscribed_Stickers_rank') > 0.2) & (col('Subscribed_Stickers_rank') < 0.5),'medium').otherwise('low')))


    main_data = main_data.withColumn('paid_sticker_usage',when(col('Can_Pay') == "No","NA").otherwise(col('paid_sticker_usage')))
    main_data = main_data.withColumn('Abilty_to_pay',when(col('Can_Pay') == "No","NA").otherwise(col('Abilty_to_pay')))
    print ds
    main_data.select('user_id_n','Can_Pay','Abilty_to_pay','Free_sticker_usage','paid_sticker_usage').write.mode('overwrite').parquet("gs://ds-url-catag/plenty_stickers_data/"+ds+"/pre_final_ds/")